In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import os

### Utilities 

In [19]:
def summarize_directory(directory,print_errors=False,obstacle="false"):
    track_lengths = {"track_porto": 57.492612883355136,"racecar_walker": 73.2483359322749,"track_barca": 221.14172044855775}
    track_names = {'track_porto':"Porto",'racecar_walker':'Walker','track_barca':"Barca"}
    controller_names = {"mpcc":"MPCC","mpc_hype":"MPC Hype","disparity_extender":"DE","pure_pursuit":"Pure Pursuit"}
    target_controllers = {"disparity_extender":"DE","pure_pursuit":"Pure Pursuit"}
    progress_file_str = "progress_{}_{}_{}_{}_{}.csv"
    collision_file_str = "collisions_{}_{}_{}_{}_{}.csv"
    start_str = directory + '/'
    cols={0:"racecar_name",1:"laps_completed",2:"race_time",3:"experiment_number"}
    rows = []

    interesting_dfs = []
    for track in track_names.keys():
        for controller in controller_names.keys():
            for opp in target_controllers.keys():
                for target in target_controllers:
                    try:
                        pdf = start_str+progress_file_str.format(track,obstacle,controller,opp,target)

                        cdf = start_str+collision_file_str.format(track,obstacle,controller,opp,target)


                        df = pd.read_csv(pdf,header=None).rename(columns=cols)

                        # some of the measurement's are not correct
                        rows_to_drop = df[(df['laps_completed'] > 0.5) & (df['race_time'] < 12)].index.values
                        df.drop(rows_to_drop)

                        # some controller's have perfect safey so there will not be a collision file
                        if(os.path.exists(cdf)):
                            df2 = pd.read_csv(cdf,header=None)[0:30]
                            val = 100 * (1-(df2.count()[0]/30))
                        else:
                            val = 100.0
                        new_df =  df.groupby("racecar_name").median()
                        new_df['efficiency'] = (new_df['laps_completed'] / new_df['race_time']) * track_lengths[track]
                        new_df = new_df.reset_index()
                        row = [track_names[track],controller_names[controller],target_controllers[opp],target_controllers[target],new_df[new_df['racecar_name'] =='racecar2']['efficiency'].values[0],new_df[new_df['racecar_name'] =='racecar']['efficiency'].values[0],new_df['race_time'][0],val]
                        rows.append(row)

                        if(new_df['efficiency'].max()>6):
                            interesting_dfs.append(df)
                    except Exception as e:
                        if(print_errors):
                            print(e)
                        continue
    return rows

### Two Car Experiments

In [20]:
rows = summarize_directory('two_vehicle_experiments')
df = pd.DataFrame(rows, columns =['Track', 'Approach','Opponent Controller','Local Planner','Ego Efficiency','Opponent Efficiency','Race Duration (s)','Safety'])
df = round(df,2)
df = df.set_index('Track').sort_values(by=["Track","Approach"])
print(df.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &      Approach & Opponent Controller & Local Planner &  Ego Efficiency &  Opponent Efficiency &  Race Duration (s) &  Safety \\
Track  &               &                     &               &                 &                      &                    &         \\
\midrule
Barca  &            DE &                  DE &            DE &            5.12 &                 4.50 &               7.63 &    0.00 \\
Barca  &            DE &                  DE &  Pure Pursuit &            5.16 &                 4.47 &               7.69 &    0.00 \\
Barca  &      MPC Hype &                  DE &            DE &            0.00 &                 5.85 &               5.47 &    0.00 \\
Barca  &      MPC Hype &                  DE &  Pure Pursuit &            0.06 &                 5.74 &               5.49 &    0.00 \\
Barca  &          MPCC &                  DE &            DE &            3.18 &                 3.01 &              11.26 &    0.00 \\
Barca  &

### Three Car Experiments

In [16]:
rows = summarize_directory('three_vehicle_experiments')
df2 = pd.DataFrame(rows, columns =['Track', 'Approach','Opponent Controller','Local Planner','Ego Efficiency','Opponent Efficiency','Race Duration (s)','Safety'])
df2 = round(df2,2)
df2 = df2.set_index('Track').sort_values(by=["Track","Approach"])
print(df2.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &      Approach & Opponent Controller & Local Planner &  Ego Efficiency &  Opponent Efficiency &  Race Duration (s) &  Safety \\
Track  &               &                     &               &                 &                      &                    &         \\
\midrule
Barca  &            DE &                  DE &            DE &            5.06 &                 4.60 &               6.09 &    0.00 \\
Barca  &            DE &                  DE &  Pure Pursuit &            5.04 &                 4.57 &               6.05 &    0.00 \\
Barca  &      MPC Hype &                  DE &            DE &            0.69 &                 5.11 &               5.37 &    0.00 \\
Barca  &      MPC Hype &                  DE &  Pure Pursuit &            0.09 &                 4.64 &               5.36 &    0.00 \\
Barca  &          MPCC &                  DE &            DE &            3.30 &                 2.70 &              10.60 &    0.00 \\
Barca  &

### Dynamic Obstacle Experiments

In [21]:
rows = summarize_directory('dynamic_obstacle_experiments',obstacle="true")
df3 = pd.DataFrame(rows, columns =['Track', 'Approach','Opponent Controller','Local Planner','Ego Efficiency','Opponent Efficiency','Race Duration (s)','Safety'])
df3 = round(df3,2)
df3 = df3.set_index('Track').sort_values(by=["Track","Approach"])
print(df3.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &      Approach & Opponent Controller & Local Planner &  Ego Efficiency &  Opponent Efficiency &  Race Duration (s) &  Safety \\
Track &               &                     &               &                 &                      &                    &         \\
\midrule
Porto &            DE &                  DE &            DE &            4.88 &                 3.78 &              24.64 &   16.67 \\
Porto &            DE &                  DE &  Pure Pursuit &            5.03 &                 4.08 &              21.21 &   10.00 \\
Porto &      MPC Hype &                  DE &            DE &            1.08 &                 4.77 &               5.39 &    0.00 \\
Porto &      MPC Hype &                  DE &  Pure Pursuit &            2.16 &                 4.42 &               9.05 &    0.00 \\
Porto &          MPCC &                  DE &            DE &            2.78 &                 5.20 &              11.32 &    0.00 \\
Porto &       

### Static Obstacle Experiments

In [28]:
rows = summarize_directory('static_obstacle_experiments',False,obstacle="false")
df4 = pd.DataFrame(rows, columns =['Track', 'Approach','Opponent Controller','Local Planner','Ego Efficiency','Opponent Efficiency','Race Duration (s)','Safety'])
df4 = round(df4,2)
df4 = df4.set_index('Track').sort_values(by=["Track","Approach"])
print(df4.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &      Approach & Opponent Controller & Local Planner &  Ego Efficiency &  Opponent Efficiency &  Race Duration (s) &  Safety \\
Track &               &                     &               &                 &                      &                    &         \\
\midrule
Porto &            DE &                  DE &            DE &            0.41 &                 1.77 &               5.51 &    0.00 \\
Porto &            DE &                  DE &  Pure Pursuit &            1.05 &                 1.74 &               5.43 &    0.00 \\
Porto &      MPC Hype &                  DE &            DE &            0.92 &                 2.13 &               5.57 &    0.00 \\
Porto &      MPC Hype &                  DE &  Pure Pursuit &            2.21 &                 0.55 &              12.27 &    0.00 \\
Porto &          MPCC &                  DE &            DE &            1.24 &                 1.46 &               7.66 &    6.67 \\
Porto &       